### Importações necessárias

In [4]:
import numpy as ny
import math
from numpy import *
from numpy.linalg import inv
import numpy.linalg as la
import csv

### Função de impressão tester

In [5]:
def pmatriz(matrizz,mens):
    matrizz=ny.array(matrizz)
    print('*='*50)
    print(f'{mens:^90}')
    print('*='*50)
    print('      ',end='')
    for i in range(1,matrizz.shape[1]+1):
        print(f' {i:^4}', end='')
    print('         ','*='*50)
    for i in range(matrizz.shape[0]):
        print(f'{i+1:>2} == ', end='')
        for j in range(matrizz.shape[1]):
            if type(matrizz[i][j]) == ny.int32:
                print(f'{matrizz[i][j]:>4} ', end='')
            else:
                print(f'{matrizz[i][j]:>6.3f} ', end='')
        print()
    print('*='*50)

### Função para cálculo dos suportes dos pontos médios dos itens de determinada Regra de Associação

In [6]:
def calc_sup(lin_ra,vetor):
    item = []
    for i in range(len(vetor[lin_ra])):
        item.append([])
        item[i].append(zpr[(vetor[lin_ra][i])-1])
    vetsup = ny.sum(item, axis=0)
    soma = 0
    for i in vetsup[0]:
        if i == len(vetor[lin_ra]):
            soma+=1
    return soma

### Criando a matriz[n][m] de respostas a partir dos dados fornecidos pelo mestre Altobelli.

In [7]:
arq = open('bloodPressureNishisatobook-base.D18.N15.txt', 'r')
texto = arq.readlines()
matriz=[]
lres=[]
nrpp = 3
for linha in texto:
    limite = 0
    linElem = linha.split()
    while limite < len(linElem):
        a = int(linElem[limite]) - nrpp * limite
        if (a % 3 == 0):
            lres += [0,0,1]
        elif (a % 2 == 0):
            lres += [0,1,0]
        else:
            #from IPython.core.debugger import Tracer; Tracer()()
            lres += [1,0,0]
        limite += 1
    matriz += [lres]
    lres=[]
arq.close()
MPR = ny.matrix(matriz)
#print(*matrizF,sep='\n')
ncol = MPR.shape[1]
nlin = MPR.shape[0]
nperguntas = ncol//nrpp
Des=ncol-nperguntas-1
#print('\nNúmero de colunas = ',ncol)
#print('Número de linhas = ',nlin)
#print('Número de perguntas = ',nperguntas)
#print('Dimensões do espaço solução = ',Des)
#pmatriz(MPR,'Matriz de Padrão de Respostas')

### Matriz transposta da MPR

In [8]:
MPRt = ny.transpose(MPR) 
zpr = ny.array(MPRt)
#print(MPRt.shape)

### Vetor de frequência de linhas (Vfl)

In [9]:
#Cálculo da Matriz Diagonal de Linhas de F
Vfl = ny.sum(MPR, axis=1)
#print (Vfl)

### Vetor de frequência de colunas (Vfcc)

In [10]:
Vfcc=[]
soma=0
for i in range(ncol):
    for j in range(len(matriz)):
        soma+=matriz[j][i]
    Vfcc.append(soma)
    soma = 0
Vfc = ny.sum(MPR, axis=0)
#print(Vfc.shape,'\n')
#print(len(Vfcc))
#print(Vfc,'\n\n',Vfcc)

### Cálculo das matrizes diagonais - MdL = matriz diagonal de linhas(nxn) - MdC = matriz diagonal de colunas(mxm)

In [11]:
MdC = ny.diagflat(Vfc)
MdL = ny.diagflat(Vfl)
#print('   Matriz Diagonal de Linhas')
#print(*MdL, sep='\n')
#print('     Matriz Diagonal de Colunas')
#print(*MdC, sep = '\n')

### Matriz de correlação entre colunas

In [12]:
MdCi = inv(MdC)
MdLi = inv(MdL)
Mcc = MPRt * MdLi * MPR * MdCi 
#pmatriz(Mcc,'Matriz de Correlação entre Colunas')

### Vetor de autovalores(Vetaval) e matriz de autovetores(Mavet) com normalização

In [13]:
vetaval,mavet=la.eig(Mcc)
Vetaval=ny.flip(ny.sort(vetaval, axis=0),0)
valaux=mavet.T.tolist() # transforma uma matriz numpy em uma lista para facilitar a manipulação dos dados.
Mavet=[] # Cria a Matriz de autovetores ordenada em função dos autovalores
for elem in Vetaval:
    ix = vetaval.tolist().index(elem)
    Mavet.append(valaux[ix])
Mavet=ny.array(Mavet).T.astype(ny.float32) # converte números complexos para float 32
Vetaval=Vetaval.astype(ny.float32)
#'\n',Vetaval)
Vetaval=ny.delete(Vetaval,0,0) # apaga a solução trivial no vetor de autovalores e na matriz
Mavet = ny.delete(Mavet,0,1)   # de autovetores removendo a primeira dimensão
while Des < Vetaval.shape[0]:
    Vetaval = ny.delete(Vetaval,Des,0)
    Mavet = ny.delete(Mavet,Des,1)

C:\Users\JRicardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  
C:\Users\JRicardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


### Cálculo do Rho

In [14]:
rho=list()
for x in Vetaval:
    rho.append(sqrt(x))
rho=ny.array(rho)
#for i in range(len(rho)):
 #   print(f'{rho[i]:>5.4f}  ', end='')

### Matriz resultante do porduto de Hadamard(MpHad)

In [15]:
MpHad=ny.multiply(Mavet,Mavet)
#print(MpHad)

### Calculando a Matriz T e seu vetor de frequência de colunas

In [16]:
matrizT = MdC * MpHad #equação(7)
matrizT = ny.array(matrizT)
VfcT=list()
VfcT=ny.sum(matrizT, axis=0) #soma coluna, caso axis=1 somaria linha = equação(8)
#for i in VfcT:
    #print(f'{i:>5.4f} ', end='')

### Vetor de multiplicadores e escalar da matriz padrão-de-respostas. eq(9)

In [17]:
escalarMPR = ny.sum(matriz)
VetMulticT = list()
for i in VfcT:
    VetMulticT.append(math.sqrt(escalarMPR/i))
#for i in range(len(VetMulticT)):
 #   print(f'{VetMulticT[i]:>7.4f}', end='')

### Cálculo da Matriz de Pesos Padrão dos itens(Mppi = x_normed)

In [18]:
# Equação(10) Matriz de pesos padrão dos itens(Mppi)
Mppi=list()
aux=list()
for i in range(Mavet.shape[0]):
    for n in range(len(VetMulticT)):
        aux.append(Mavet[i][n]*VetMulticT[n])
    Mppi.append(aux[:])
    aux.clear()
Mppi = ny.array(Mppi)
#pmatriz(Mppi,'Matriz de Pesos Padrão dos Itens')

### Matriz das Coordenadas dos Itens nas Dimensões (Mcides = x_projected)

In [19]:
Mcides = Mppi * rho
#pmatriz(Mcides,'Matriz de Coordenadas dos Itens nas Dimensões')

### Matriz das Coordenadas das Transações nas Dimensões

In [20]:
Ctdes = MPR * Mppi # fórmula (13)
Ctdes = ny.array(Ctdes)
#pmatriz(Ctdes,'Matriz de Coordenadas das Transações nas Dimensões')

### Matriz de Pesos Padrão das Transações (Mppt = y_normed)

In [21]:
aux= Vfl * rho
Mppt = Ctdes/aux # fórmula (14)
#pmatriz(Mppt,'Matriz de Pesos Padrão de Transações')

### Matriz de Coordenadas das Transações nas Dimensões (Mctd = y_projected)

In [22]:
Mctd = Ctdes / Vfl # fórmula (15) simplificada
Mctd = ny.array(Mctd)
#pmatriz(Mctd,'Matriz de Coordenadas das Transações de itens')

### Calculando o Delta

In [23]:
Delta = Vetaval * (100 / ny.sum(Vetaval))
#for i in Delta:
 #   print(f'{i:5.4f}  ', end='')

### Função para cálculo do ponto médio dos pontos fornecidos pelo arquivo bloodPressureNishisatobook.D18.N15.txt

In [24]:
def calcpm(ptos):
    a = ptos
    soma = ny.zeros(Des, dtype=float32)
    for i in range(Des):
        for j in range(len(a)):
            soma[i]+= Mcides[a[j]-1,i]
            #print(f'{a[j]} {soma[i]:.2f}')
        soma[i]=round(soma[i]/len(a),3)
        #print(soma[i])
    #print(soma)
    return soma

### Preparando a montagem do arquivo de extensão CSV

In [25]:
arq2 = open('bloodPressureNishisatobook.D18.N15.txt', 'r') # ARQUIVO QUE CONTÉM AS REGRAS DE ASSOCIAÇÃO DE INTERESSE
texto2 = arq2.readlines()
arq2.close()
confList = []
confDesejada = 0 # 40% de confiança
for x in texto2:
    linhatxt = x.split()
    conflinha = float(linhatxt[len(linhatxt)-1])
    if conflinha >= confDesejada:
        confList.append(linhatxt[:])
antcd = []
consq = []
sup = []
aux1 = []
aux2 = []
pm1=[]
pm2=[]
for y in confList:
    ind = y.index('==>') 
    sup.append(y[y.index('#SUP:')+1])
    for ini in range(len(y)):
        if y[ini]=='#SUP:':
            consq.append(aux2[:])
            aux2.clear()
            ini=len(y)
            break
        elif ini < ind:
            aux1.append(int(y[ini]))
            next
        elif y[ini]=='==>':
            if len(aux1) > 1:
                antcd.append(aux1[:])
            else:
                antcd.append(aux1[:])
            aux1.clear()
            next
        elif ini > ind:
            aux2.append(int(y[ini]))
for i in range(len(confList)):
    confianca = float(confList[i][len(confList[i])-1])*100
    pm1.append(calcpm(antcd[i])) # pontos médios de todos antecedentes
    pm2.append(calcpm(consq[i])) # pontos médios de todos consequentes
pm1 = ny.array(pm1)
pm2 = ny.array(pm2)
    #if len(antcd[i])>= 2 and len(consq[i])>= 2:
    #print(f'projeção de {antcd[i]} em {consq[i]} suporte = {sup[i]} e confiança = {confianca:5.1f}% pm1 {pm1[i]} pm2 {pm2[i]}.')
#for y in confList:
 #   a=float(y[len(y)-1])*100
  #  print(f'Confiança de {a:5.1f}%')
#for i in range(len(confList)):
    #print(pm1[i],' e ', pm2[i])
#print(total)

### Sem função definida até agora(9/5/2019 ; 1:59hs

In [26]:
#a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
#result = dist2ptsi(a[:])
#pmatriz(result,'Distância entre todos os itens')
d_2_ptos = ny.zeros(len(pm1), dtype=float32)
for i in range(len(pm1)):
    soma = 0
    for j in range(pm1.shape[1]):
        soma += (pm1[i][j]-pm2[i][j])**2
    d_2_ptos[i] = math.sqrt(soma)
#c = [4,7,8]
#dist2ptsi(c[:], Mcides, Vfcc, Des, rho, nlin)

### Função para cálculo da distância entre itens (modelo 1)

In [27]:
def dist2ptsi(ptos, M_cides, V_fc, tamanho, roh, n_lin):
    ematriz = (len(ptos) == M_cides.shape[0])
    print(ematriz)
    if ematriz: Ditens = ny.zeros((len(ptos),len(ptos)), dtype=float32) # matriz de zeros
    if len(ptos) < 2:
        return 'Hove um erro e foi passado apenas um ponto!'
    while len(ptos) >= 2:
        while i in range(len(ptos)):
            print('aqui',i)
            for j in range(1,len(ptos)):
                d_itens = 0
                print(ptos[0],ptos[j])
                linha = ptos[0]-1
                coluna = ptos[j]-1
                for k in range(tamanho):
                    d_itens += roh[k] * (((M_cides[linha][k])/math.sqrt(V_fc[linha]/n_lin) 
                              - (M_cides[coluna][k])/math.sqrt(V_fc[coluna]/n_lin))**2) #Qui-quadrado
                if ematriz:
                    Ditens[linha,coluna] = round(d_itens,2)
                    Ditens[coluna,linha] = round(d_itens,2)
                else: print(f'Distância entre {ptos[i]} e {ptos[j]} = {d_itens:.2f}')
            ptos.pop(0)
        print(len(ptos))
    if ematriz: return Ditens

### Função para cálculo da distância entre itens (modelo 2 - submatrizes de distâncias dada det. regra de associação)

In [28]:
def dist2ptsi2(ptos, M_cides, V_fc, tamanho, roh, n_lin):
    Ditens = ny.zeros((len(ptos),len(ptos)), dtype=float32) # matriz de zeros
    i=0
    while i in range(len(ptos)):
        for j in range(len(ptos)):
            d_itens = 0
            linha1 = ptos[i]-1
            linha2 = ptos[j]-1
            for k in range(tamanho):
                d_itens += roh[k] * (((M_cides[linha1][k])/math.sqrt(V_fc[linha1]/n_lin) 
                            - (M_cides[linha2][k])/math.sqrt(V_fc[linha2]/n_lin))**2) #Qui-quadrado
            if i != j:
                Ditens[i,j] = round(d_itens,2)
                Ditens[j,i] = round(d_itens,2)
        i+=1
    return Ditens

### Calculando o suporte do ponto médio antecedente e consequente

In [29]:
supant = []
supcons = []
for i in range(len(antcd)):
    supant.append(calc_sup(i,antcd))
    supcons.append(calc_sup(i,consq))
    #print('linha = ',i+1, 'sup antec.',antcd[i],' = ',supant[i],' === sup conseq.',consq[i],' = ',supcons[i])

### Função para cálculo da distância entre os pontos médios antecedente e consequente

In [30]:
def calcdistfim(roh, px1, px2, sup1, sup2, tam, ditens):
    for k in range(len(px1)):
        a = px1[k]
        b = px2[k]
        c = math.sqrt(sup1/tam)
        d = math.sqrt(sup2/tam)
        ditens += roh[k] * ((a/c 
                            - b/d)**2) #Qui-quadrado
    #print(d_itens)
    return ditens
    

### Cálculo do vetor de distância final entre os pontos médios antecedentes e consequentes

In [46]:
distfinal = []
for i in range(len(antcd)):
    a = calcdistfim(rho, pm1[i], pm2[i], supant[i], supcons[i], nlin, 0)
    distfinal.append(a)
origem = ny.array([[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]])
dist_antecd_origem = []
for i in range(len(antcd)):
    a = calcdistfim(rho, origem, pm1[i], supant[i], supcons[i], nlin, 0)
    dist_antecd_origem.append(a)
dist_conseq_origem = []
for i in range(len(antcd)):
    a = calcdistfim(rho, origem, pm2[i], supant[i], supcons[i], nlin, 0)
    dist_conseq_origem.append(a)
for i in range(len(dist_conseq_origem)):
    print('Para ',i,' = ',dist_conseq_origem[i])

Para  0  =  [3.80871545]
Para  1  =  [2.13576016]
Para  2  =  [3.20364024]
Para  3  =  [4.66338519]
Para  4  =  [3.20364024]
Para  5  =  [4.6084368]
Para  6  =  [3.20364024]
Para  7  =  [3.51439693]
Para  8  =  [3.20364024]
Para  9  =  [1.71979025]
Para  10  =  [3.20364024]
Para  11  =  [3.08576537]
Para  12  =  [2.13576016]
Para  13  =  [1.71979025]
Para  14  =  [2.13576016]
Para  15  =  [2.12558509]
Para  16  =  [2.13576016]
Para  17  =  [3.02112455]
Para  18  =  [2.13576016]
Para  19  =  [3.079184]
Para  20  =  [4.24713047]
Para  21  =  [1.71979025]
Para  22  =  [3.51439693]
Para  23  =  [4.29277647]
Para  24  =  [3.51439693]
Para  25  =  [2.12558509]
Para  26  =  [3.62080658]
Para  27  =  [1.71979025]
Para  28  =  [1.71979025]
Para  29  =  [4.37516694]
Para  30  =  [1.71979025]
Para  31  =  [3.02112455]
Para  32  =  [1.71979025]
Para  33  =  [3.08576537]
Para  34  =  [2.12558509]
Para  35  =  [3.08576537]
Para  36  =  [2.12558509]
Para  37  =  [3.079184]
Para  38  =  [3.80871545]
P

Para  499  =  [3.59464567]
Para  500  =  [7.29783652]
Para  501  =  [6.76901375]
Para  502  =  [6.77145125]
Para  503  =  [6.67571379]
Para  504  =  [5.086519]
Para  505  =  [4.628898]
Para  506  =  [3.80871545]
Para  507  =  [2.13576016]
Para  508  =  [4.24713047]
Para  509  =  [1.71979025]
Para  510  =  [2.12558509]
Para  511  =  [3.079184]
Para  512  =  [3.00318094]
Para  513  =  [5.02253755]
Para  514  =  [2.50674269]
Para  515  =  [2.95495796]
Para  516  =  [4.35281475]
Para  517  =  [3.55568771]
Para  518  =  [1.69726378]
Para  519  =  [2.03529873]
Para  520  =  [2.87632702]
Para  521  =  [3.14588893]
Para  522  =  [3.4464494]
Para  523  =  [5.2368235]
Para  524  =  [1.22955091]
Para  525  =  [2.17425125]
Para  526  =  [3.02359191]
Para  527  =  [3.99067405]
Para  528  =  [3.04451723]
Para  529  =  [2.96972118]
Para  530  =  [3.87499759]
Para  531  =  [4.16087365]
Para  532  =  [4.44366339]
Para  533  =  [5.71932934]
Para  534  =  [2.58772613]
Para  535  =  [3.15384016]
Para  536

Para  923  =  [4.33484598]
Para  924  =  [6.89127414]
Para  925  =  [4.36492031]
Para  926  =  [2.59729336]
Para  927  =  [6.37857884]
Para  928  =  [4.38274718]
Para  929  =  [4.53369578]
Para  930  =  [3.02049351]
Para  931  =  [6.75460659]
Para  932  =  [6.06810917]
Para  933  =  [3.72986423]
Para  934  =  [5.26129579]
Para  935  =  [3.43657125]
Para  936  =  [6.6370247]
Para  937  =  [4.8810158]
Para  938  =  [3.49651197]
Para  939  =  [4.12636181]
Para  940  =  [3.20364024]
Para  941  =  [4.6084368]
Para  942  =  [3.51439693]
Para  943  =  [4.91157985]
Para  944  =  [2.12558509]
Para  945  =  [3.08576537]
Para  946  =  [4.34121796]
Para  947  =  [3.67016948]
Para  948  =  [5.25622539]
Para  949  =  [2.46745427]
Para  950  =  [3.62715415]
Para  951  =  [5.3233326]
Para  952  =  [5.48199277]
Para  953  =  [4.93364103]
Para  954  =  [5.18532821]
Para  955  =  [5.29645685]
Para  956  =  [3.13327954]
Para  957  =  [4.56054103]
Para  958  =  [5.10533614]
Para  959  =  [7.37700915]
Para 

### Uso da função para cálculo da distância entre itens e cálculo da distância entre dois pontos quaisquer da lista passada através do arquivo bloodPressureNishisatobook.D18.N15.txt usado na montagem do arquivo .csv

In [32]:
Mdara = []
Mdcra = []
for lina in range(len(antcd)):
    Mdara.append([])
    Mdcra.append([])
    aux1 = dist2ptsi2(antcd[lina], Mcides, Vfcc, Des, rho, nlin)
    aux1 = ny.array(aux1, dtype=float32)
    Mdara[lina].append(aux1)
    aux2 = dist2ptsi2(consq[lina], Mcides, Vfcc, Des, rho, nlin)
    aux2 = ny.array(aux2, dtype=float32)
    Mdcra[lina].append(aux2)
soma = 0

### Testando a nova meta imposta pelo mestre Altobelli (matrizes do antecedente e consequente)

In [47]:
'''arq3 = open('itens.csv', 'w', encoding='utf-8', newline='')
try:
    escreve = csv.writer(arq3, delimiter=';')
    escreve.writerow(['antecedente','consequente','suporte','confiança', 'pto médio ant.', 'pto médio cons.', 'distância entre os ptos.', 'matriz dist antecedente', 'matriz dist consequente'])
    for i in range(len(confList)):
        a = antcd[i],consq[i],sup[i],confList[i][len(confList[i])-1],pm1[i],pm2[i],d_2_ptos[i], Vdara[i][0], Vdcra[i][0]
        escreve.writerow(a)
finally:
    arq3.close()
    
arq4 = open('itens2.csv', 'w', encoding='utf-8', newline='')
try:
    escreve = csv.writer(arq4, delimiter=';')
    escreve.writerow(['antecedente', 'consequente', 'suporte', 'confiança', 'distância entre os ptos.',
                      'matriz dist antecedente', 'matriz dist consequente'])
    for i in range(len(confList)):
        a = antcd[i], consq[i], sup[i], confList[i][len(confList[i])-1], d_2_ptos[i], Vdara[i][0], Vdcra[i][0]
        escreve.writerow(a)
finally:
    arq4.close()
    
arq6 = open('itens_finales.csv', 'w', encoding='utf-8', newline='')
try:
    escreve = csv.writer(arq6, delimiter=';')
    escreve.writerow(['Antecedente', 'Consequente', 'Suporte', 'Confiança'])
    for i in range(len(confList)):
        confianca = str(round(float(confList[i][len(confList[i])-1])*100,1))+'%'
        b = antcd[i], consq[i], sup[i], confianca
        escreve.writerow(b)
finally:
    arq6.close()'''

arq5 = open('itens_final.csv', 'w', encoding='utf-8', newline='')
try:
    escreve = csv.writer(arq5, delimiter=';')
    escreve.writerow(['Antecedente', 'Consequente', 'Suporte', 'Confiança', 'Matriz Antecedentes',
                      'Matriz Consequentes', 'pto médio ant.', 'pto médio cons.', 'suporte antec.', 'suporte conseq.', 'distância final','antecedete x origem', 'consequente x origem'])
    for i in range(len(confList)):
        confianca = str(round(float(confList[i][len(confList[i])-1])*100,1))+'%'
        b = antcd[i], consq[i], sup[i], confianca, Mdara[i][0], Mdcra[i][0],pm1[i],pm2[i],supant[i],supcons[i],distfinal[i],dist_antecd_origem[i],dist_conseq_origem[i]
        escreve.writerow(b)
finally:
    arq5.close()

### Função para cálculo da distância entre transações

In [35]:
def dist2ptst(ptos):
    ematriz = (len(ptos) == Mctd.shape[0])
    if ematriz: 
        print('é matriz')
        Ditens = ny.zeros((len(ptos),len(ptos)), dtype=float32) # matriz de zeros
    if len(ptos) < 2:
        return 'Hove um erro e foi passado apenas um ponto!'
    i=0
    while len(ptos) >= 2:
        while i in range(len(ptos)):
            for j in range(1,len(ptos)):
                d_itens = 0
                linha = ptos[0]-1
                coluna = ptos[j]-1
                for k in range(Des):
                    d_itens += rho[k] * (((Mctd[linha][k]/math.sqrt(Vfl[linha]/ncol))
                                          - (Mctd[coluna][k]/math.sqrt(Vfl[coluna]/ncol)))**2) #Qui-quadrado
                if ematriz:
                    Ditens[linha,coluna] = round(d_itens,2)
                    Ditens[coluna,linha] = round(d_itens,2)
                else: print(f'Distância entre {ptos[i]} e {ptos[j]} = {d_itens:.2f}')
            ptos.pop(0)
    if ematriz: return Ditens

### Cálculo da distância entre as transações

In [42]:
'''listptst=list() # lista de regras de confiança para projeção com mais de um ponto.
for i in range(len(pem)):
    if (len(pem[i]))>=2:
        for aux in [i]:
            listptst += [aux]'''          
#for i in listptst:
    #print(f'Para o vetor {pem[i]}, temos:')
    #dist2ptst(pem[i][:])
    #print()
#b = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#pmatriz(dist2ptst(b[:]),'Distância entre todos os itens')
#c = [4,7,8]
#dist2ptst(c[:])
print()

1


TypeError: __array_prepare__ must return an ndarray or subclass thereof which is otherwise identical to its input

### Cálculo da distância entre os itens e as transações

In [ ]:
MDIeT = ny.zeros((len(Vfcc),len(Vfl)), dtype=float32)
for item in range(len(Vfcc)):
    for transacao in range(len(Vfl)):
        dist = 0
        for dimensao in range(Des):
            a1 = Mctd[transacao][dimensao]
            a2 = Vfl[transacao] / ncol
            b1 = Mcides[item][dimensao]
            b2 = Vfcc[item] / nlin
            a = a1**2 / a2
            b = b1**2 / b2
            c = a1 * b1 / sqrt(a2 * b2)
            dist += rho[dimensao] * (a + b - 2 * sqrt(rho[dimensao]) * c)
        MDIeT[item, transacao] = dist
#pmatriz(MDIeT,'Matriz de distâncias entre Itens e Transações')

#### Ponto médio da distância de todos os itens do grupo para cada uma das transações

In [ ]:
Vetpmit = ny.sum(MDIeT, axis=0)/MDIeT.shape[0]
#print(Vetpmit)

Atividades do TCC
Esqueleto - entrar no cederj onde tem um template do TCC no word.
Trabalhos Relacionados.
Atividade. Relevância no que se deseja pesquisar -  regra de associação 
Dual scalling é uma ferramenta visual para ajudar representar as regras de associação.
Evidenciar a importancia das regras de associação.
Selecionar na literatura alguns trabalhos que utilizam a RA para alguma finalidade prática.
RA - trabalhos relacionados. Google escola
3 ou 4 trabalhos - No trabalho tal o autor tal desenvolve-se em função ...
Visando entender a aplicabilidade das RAs nas necessidades do mundo contemporâneo, utilizam 
a fim com intuito de ...
Analista de dados tomar decidões ou prever comportamentos a partir das RAs.
Fundamentação teórica.
Visualizador.
Testes ou Resultados.
Conclusão.
